In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-3.5-turbo' 
MODEL = 'llama2'

from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(api_key = OPENAI_API_KEY, model = MODEL)

model.invoke("who is Shruti Haasan") #AI message



In [ ]:
# code to remove AI message of Chat Gpt
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("who is shruti haasan?")

In [56]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = 'llama2'

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model = MODEL)
embeddings = OllamaEmbeddings()



In [57]:
model.invoke("who is Shruti Haasan")

'Shruti Haasan is an Indian actress, singer, and film producer who works primarily in Telugu, Tamil, and Hindi films. She is the daughter of actors Kamal Haasan and Sarika Thakur, and has been active in the film industry since 2009.\n\nShruti Haasan was born on January 12, 1986, in Chennai, Tamil Nadu, India. She began her acting career at a young age, making her debut as a child artist in the 1999 film "Hey Ram." She went on to act in several films throughout her childhood and adolescence, including "Lakshya" (2004), "Dil K Ab Bas Ye Hai" (2005), and "Anbe Aaruyire" (2005).\n\nAfter completing her higher secondary education, Shruti pursued a career in singing. She released her debut album, "Bheg," in 2015, which was well-received by audiences and critics alike. She has since released several other albums, including "Oh My Kadavule" (2018) and "Unbleached" (2020).\n\nIn addition to her acting and singing career, Shruti Haasan is also a film producer. She co-founded the production compa

In [64]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader



In [65]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings



# Define your documents
loader = PyPDFLoader('results.pdf')
docs = loader.load_and_split()

# Initialize vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Use the vector store as needed


In [68]:
retriever = vectorstore.as_retriever()
retriever.invoke("Shaurya Agarwal")

[Document(metadata={'source': 'results.pdf', 'page': 3}, page_content="Paper Code (Credit***)\nInternal\nTotal (Grade**)LEGEND\nExternal\n** If Grade Based*** If Credit Based\nProgramme Code: 128      Programme Name: BACHELOR OF TECHNOLOGY  (ELECTRONICS AND COMMUNICATIONS ENGINEERING)      Sem./Year/EU: EIGHTH SEMESTER      Batch: 2019      Examination: REGULAR JUL, 2023    Result Declared Date :31-JUL-23\nS.No. Photo Student's Details Institution Code: 164      Institution: UNIVERSITY SCHOOL OF INFORMATION, COMMUNICATION & TECHNOLOGY (FORMERLY USIT) CS/Remarks\nHS402 (2)\n-\n79 (A+)79MS416 (4)\n22\n83 (A+)61HS424 (1)\n-\n70 (A)70EC426 (4)\n20\n82 (A+)62EC452 (2)\n32\n76 (A+)44EC454 (8)\n37\n94 (O)57EC456 (1)\n-\n93 (O)9302216412819\nABHISHEK JAIN\nSID: 190000117699\nSchemeID: 191282015001\n 21\n22\nHS402 (2)\n-\n71 (A)71MS416 (4)\n16\n65 (A)49HS424 (1)\n-\n72 (A)72EC426 (4)\n19\n79 (A+)60EC452 (2)\n30\n75 (A+)45EC454 (8)\n35\n89 (A+)54EC456 (1)\n-\n84 (A+)8402316412819\nSACHIN KUMAR\n

In [39]:
from langchain.prompts import PromptTemplate
template =  '''Answer the  question based on the context below. If you can't answer
the question, reply "I don't know"

Context: {context}
Question: {question}
'''

prompt = PromptTemplate.from_template(template)
print(prompt.format(context = 'Here is some context', question = "Here is  some question")) 



Answer the  question based on the context below. If you can't answer
the question, reply "I don't know"

Context: Here is some context
Question: Here is  some question



In [40]:
chain = prompt | model

In [41]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [42]:
chain.invoke(
    {
        'context':  "I was 10 years old in 2020",
        'question': "what's my age in 2024"
    }
)

'Sure! Based on the information provided, your age in 2024 would be 14 years old.'

In [71]:
from operator import itemgetter
chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question')
    }
    | prompt
    | model
)

In [79]:
res = chain.invoke({"question": "Show the result of 'Ragini Bhayana'?"})

In [80]:
print(res)

Sure! Here is the result for "Ragini Bhayana":

SID: 190000117684
SchemeID: 191282015001

* Examination Unit Number: 7400716412819
* Surname: Bhayana
* Given Name: Ragini
* Roll Number: 190000117684
* Result: Passed with grace marks (A+)

Grade: A+
Credit Secured: 1
Examination Unit Number: 7400716412819


In [82]:
for s in chain.stream({"question": "Show the result of 'Ragini Bhayana'?"}):
    print(s, end='', flush=True)

The result for "Ragini Bhayana" is:

SID: 190000117686
SchemeID: 191282015001

* Result: Passed with grace marks
* Subjects:
	+ HS402 (2)
	+ HS424 (1)
	+ EC452 (2)
	+ EC454 (8)
	+ EC456 (1)

Marks Obtained:

* HS402: 22
* HS424: 17
* EC452: 34
* EC454: 8
* EC456: 30

Total Marks: 195
Total Obtained: 179 (75%)
Grace Marks: +56
Final Result: Pass

In [83]:
questions = [
    "what is the purpose of the course?",
    "how many hours of live sessions?",
    "How much does the program costs?"
]

In [84]:
# batching the questions:
chain.batch([{"question": q} for q in questions])

['The course\'s purpose is not explicitly stated in the provided text. However, based on the information provided, it appears that the course is a professional development program for teachers, as it mentions "SchemeID" and "Examination Unit Number," which are likely references to educational institutions and examinations. The course may aim to enhance the teaching skills and knowledge of its participants, preparing them to handle various responsibilities and challenges in their profession.',
 "The document provides information about the students' performance in various subjects and their overall percentage. Here are some observations based on the data provided:\n\n1. Total number of hours attended: The document mentions that the total number of hours attended by the students is 426 hours.\n2. Live sessions attended: Out of the total hours attended, 21 hours were live sessions.\n3. Pass percentage: The overall percentage of passed students is 78%, with 42 students passing out of a tota